In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score, accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
import json
from pypmml import Model
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
import pickle

In [2]:
df = pd.read_csv('adult.data', sep=',' , names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income'])

In [3]:
df['workclass'] = df.columns.str.replace(' ', '')

In [15]:
df['workclass'][0]

' State-gov'

In [4]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

In [5]:
X.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [6]:
X.iloc[0,:]

age                           39
workclass              State-gov
fnlwgt                     77516
education              Bachelors
education-num                 13
marital-status     Never-married
occupation          Adm-clerical
relationship       Not-in-family
race                       White
sex                         Male
capital-gain                2174
capital-loss                   0
hours-per-week                40
native-country     United-States
Name: 0, dtype: object

In [7]:

label_encoder = {}
for col in X.columns:
    if type(X.loc[0 , col]) == str:
        label_encoder[col] = LabelEncoder()
        X.loc[: , col] = label_encoder[col].fit_transform(X.loc[:,col])


/tmp/ipykernel_7790/1994870652.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[: , col] = label_encoder[col].fit_transform(X.loc[:,col])


In [8]:
#Y = label_encoder.fit_transform(Y)

In [9]:
pickle.dump(label_encoder , open('encoder.pkl','wb'))

In [10]:
X_train , X_test , Y_train, Y_test = train_test_split(X,Y , test_size=0.2)


In [11]:
model = DecisionTreeClassifier()


pipeline = PMMLPipeline([
	("classifier", DecisionTreeClassifier())
])
pipeline.fit(X_train, Y_train)

# Save the fitted model
sklearn2pmml(pipeline, "dtr.pmml", with_repr = True)

# Load the saved model and predict data
model = Model.fromFile('dtr.pmml')


In [12]:
Y_train

1510      <=50K
21027      >50K
24490     <=50K
27032      >50K
4006       >50K
          ...  
32302      >50K
19280      >50K
26603      >50K
15937     <=50K
28981     <=50K
Name: income, Length: 26048, dtype: object

In [13]:
y_pred = model.predict(X_test)

In [14]:
x = label_encoder.fit_transform(Y_test)

AttributeError: 'dict' object has no attribute 'fit_transform'

In [ ]:
y = np.argmax(y_pred , axis=1)

In [ ]:
accuracy_score(x,y)

0.5722401351143866

In [ ]:

Y_test.shape
Y_test

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
Y_test = label_encoder.inverse_transform(Y_test)

In [ ]:
y_pred = label_encoder.inverse_transform(y_pred)

ValueError: y should be a 1d array, got an array of shape (6513, 2) instead.

In [ ]:
Y_test = pd.DataFrame(Y_test)
y_pred = pd.DataFrame(y_pred)

In [ ]:
total_answer = pd.concat([y_pred , Y_test])

/tmp/ipykernel_6021/1436144399.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  total_answer = pd.concat(y_pred , Y_test)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"